# Data Analysis RAG Dashboard (Streamlit Version)

This notebook is a portfolio version of the Streamlit application for data analysis.
Each step is explained in markdown so that reviewers on GitHub can understand it.

---


## 1. Import Library and Setup
We start by importing the libraries used in the original Streamlit application, including pandas, numpy, seaborn, matplotlib, statsmodels, and others.

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import openai
from fpdf import FPDF
import tempfile
from sklearn.preprocessing import LabelEncoder

## 2. Column Type Detection Function
The `detect_col_type` function will automatically detect which columns are numeric, categorical, or date-based, to facilitate automatic analysis.

In [ ]:
def detect_col_type(df):
    dtypes = {}
    force_numeric = ['bounce_flag', 'revenue_$', 'conversion_flag',
                     'pages_visited', 'time_spent', 'demographic_age']
    for c in df.columns:
        series = df[c]
        if c.lower() in [f.lower() for f in force_numeric]:
            dtypes[c] = 'numeric'
            continue
        name_hint = any(x in c.lower() for x in ['date', 'timestamp'])
        looks_like_date = series.astype(str).str.contains(r'[-/:]').mean() > 0.5
        if name_hint or looks_like_date:
            try:
                parsed = pd.to_datetime(series, errors='coerce', infer_datetime_format=True)
                if parsed.notna().sum() > len(series) * 0.5:
                    df[c] = parsed
                    dtypes[c] = 'date'
                    continue
            except:
                pass
        if pd.api.types.is_numeric_dtype(series):
            dtypes[c] = 'numeric'
        else:
            dtypes[c] = 'categorical'
    return dtypes

## 3. Statistical Test Functions
Includes automatic detection of T-tests, Mann-Whitney tests, ANOVA/Kruskal tests, regression tests, and Chi-Square tests.

In [ ]:
def detect_test(df, grp, val):
    groups = df[grp].dropna().unique()
    vals = [df[df[grp]==g][val].dropna() for g in groups]
    normal = all(stats.shapiro(v)[1] > 0.05 for v in vals if len(v)>=3)
    eq = stats.levene(*vals)[1] > 0.05
    if len(groups)==2:
        return "t-test" if normal and eq else "mann-whitney"
    return "anova" if normal and eq else "kruskal"

def run_tests(df, grp, val):
    test = detect_test(df, grp, val)
    if test in ["t-test", "mann-whitney"]:
        g = df[grp].dropna().unique()
        a,b = df[df[grp]==g[0]][val], df[df[grp]==g[1]][val]
        if test=="t-test":
            t,p = stats.ttest_ind(a,b)
            return f"T-test: t={t:.3f}, p={p:.4f}", test
        else:
            u,p = stats.mannwhitneyu(a,b)
            return f"Mann-Whitney U={u:.3f}, p={p:.4f}", test
    elif test=="anova":
        model = ols(f'{val} ~ C({grp})', df).fit()
        aov = sm.stats.anova_lm(model, typ=2)
        tuk = pairwise_tukeyhsd(df[val], df[grp])
        return f"ANOVA:\n{aov}\n\nTukey HSD:\n{tuk.summary()}", test
    else:
        vals = [df[df[grp]==g][val] for g in df[grp].unique()]
        h,p = stats.kruskal(*vals)
        return f"Kruskal-Wallis H={h:.3f}, p={p:.4f}", test

def run_chi2(df, col1, col2):
    tab = pd.crosstab(df[col1], df[col2])
    chi, p, _, _ = stats.chi2_contingency(tab)
    return f"Chi-Square: χ²={chi:.3f}, p={p:.4f}", chi, p

def run_regression(df, y, x):
    y_data = df[y]
    X_data = df[x]
    if not pd.api.types.is_numeric_dtype(y_data):
        le_y = LabelEncoder()
        y_data = le_y.fit_transform(y_data)
    for col in x:
        if not pd.api.types.is_numeric_dtype(df[col]):
            le_x = LabelEncoder()
            X_data[col] = le_x.fit_transform(df[col])
    X = sm.add_constant(X_data)
    if len(np.unique(y_data)) == 2:
        model = sm.Logit(y_data, X).fit(disp=False)
    else:
        model = sm.OLS(y_data, X).fit()
    return model.summary()

## 4. Analisis Power (Sample Size Estimation)
This section calculates the sample size per group based on Cohen's d, before the user performs statistical tests.

In [ ]:
from statsmodels.stats.power import TTestIndPower
n = TTestIndPower().solve_power(effect_size=eff, alpha=0.05, power=0.8)

## 5. Main Analysis (Statistical Test)
Users can select targets and predictors, and the system will automatically select the appropriate statistical test.

In [ ]:
st.markdown("## 5. Main Statistical Analysis (Statistical Test)")

col1 = st.selectbox("Target Column (Dependent Variable)", df.columns)
col2 = st.selectbox("Group/Predictor Column (Independent Variable)", df.columns)

target_type = dtypes[col1]
group_type = dtypes[col2]

# Tentukan uji otomatis berdasarkan tipe data
default_test = "Regression"
if target_type == 'numeric' and group_type == 'categorical':
    n_groups = df[col2].nunique()
    default_test = "T-test / Mann-Whitney" if n_groups == 2 else "ANOVA / Kruskal-Wallis"
elif target_type == 'categorical' and group_type == 'categorical':
    default_test = "Chi-Square"

test_choice = st.selectbox(
    "Choose Statistical Test (auto-selected)",
    ["T-test", "Mann-Whitney", "ANOVA", "Kruskal-Wallis", "Chi-Square", "Regression"],
    index=["T-test", "Mann-Whitney", "ANOVA", "Kruskal-Wallis", "Chi-Square", "Regression"].index(default_test.split()[0])
)

st.info(f"Auto-selected: **{default_test}** (Target: {col1} [{target_type}], Group: {col2} [{group_type}])")

# Jalankan analisis ketika tombol ditekan
if st.button("Run Analysis", use_container_width=True):
    if test_choice in ["T-test", "Mann-Whitney", "ANOVA", "Kruskal-Wallis"] and target_type == 'numeric' and group_type == 'categorical':
        res, test = run_tests(df, col2, col1)
        insight = explain(res)
        save_result('Statistical Test', text=res, insight=insight)

    elif test_choice == "Chi-Square" and target_type == 'categorical' and group_type == 'categorical':
        res, chi, p = run_chi2(df, col1, col2)
        insight = explain(res)
        save_result('Chi-Square Test', text=res, insight=insight)

    else:
        summary = run_regression(df, col1, [col2])
        save_result('Regression', text=str(summary))

# Tampilkan hasil analisis
show_result_inline('Statistical Test')
show_result_inline('Chi-Square Test')
show_result_inline('Regression')

## 6. Trend Analysis (Time-Series)
Displays trends based on date columns and numeric values, with aggregation (mean, sum, etc.).

In [ ]:
def plot_trend(df, date_col, val_col, agg_func, sort_order):
    if not pd.api.types.is_datetime64_any_dtype(df[date_col]):
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df[date_col] = df[date_col].dt.floor('D')
    agg_df = df.groupby(date_col)[val_col].agg(agg_func).sort_values(
        ascending=(sort_order == "Ascending")
    )
    fig, ax = plt.subplots(figsize=(10, 4))
    sns.lineplot(x=agg_df.index, y=agg_df.values, ax=ax, color="teal", marker="o")
    ax.set_title(f"Trend of {val_col} by Date ({agg_func})")
    ax.set_xlabel("Date")
    ax.set_ylabel(val_col)
    plt.xticks(rotation=45)
    return agg_df, fig

## 7. Categorical Aggregation
Displays the average, count, or sum by category, visualized as a barplot.

In [ ]:
def aggregate_categorical(df, cat_col, val_col, agg_func, sort_order):
    ascending = (sort_order == "Lowest")
    agg_df = df.groupby(cat_col)[val_col].agg(agg_func).sort_values(ascending=ascending)
    fig, ax = plt.subplots(figsize=(10,4))
    sns.barplot(x=agg_df.index, y=agg_df.values, ax=ax, palette="viridis")
    ax.set_title(f"{agg_func.title()} by {cat_col}")
    ax.tick_params(axis='x', rotation=45)
    return agg_df, fig

## 8. Q&A (AI Insight)
Using GPT to explain results or provide business action recommendations.

In [ ]:
def explain(text, question=None):
    if not openai_key: return "No API key."
    openai.api_key = openai_key
    prompt = f"Jelaskan hasil statistik ini untuk tim non-teknis:\n{text}"
    if question:
        prompt += f"\nJawab pertanyaan: {question}"
    res = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role":"user","content":prompt}],
        temperature=0.4
    )
    return res.choices[0].message.content

## 9. Export to PDF
All analysis results and visualizations can be exported to PDF for reporting purposes.

In [ ]:
def export_pdf(results_dict):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, "Analysis Report")

    for section, content in results_dict.items():
        pdf.ln(5)
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, section, ln=True)
        pdf.set_font("Arial", size=11)

        if isinstance(content, tuple):
            text = content[0] if len(content) > 0 else ""
            insight = content[1] if len(content) > 1 else ""
            fig = content[2] if len(content) > 2 else None

            pdf.multi_cell(0, 8, text)
            if insight:
                pdf.multi_cell(0, 8, "\nInsights:\n" + insight)
            if fig and isinstance(fig, plt.Figure):
                with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_img:
                    fig.savefig(tmp_img.name, bbox_inches="tight")
                    tmp_img.flush()
                    pdf.ln(5)
                    pdf.image(tmp_img.name, w=170)
        else:
            pdf.multi_cell(0, 8, str(content))

    tmp_pdf = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    pdf.output(tmp_pdf.name)
    return tmp_pdf.name

## 10. Conclusion

This project demonstrates how to build a modular and interactive data analysis tool using Python, integrating statistical testing, visualization, and AI-powered explanations into a single workflow.

Key takeaways:
- The app automatically detects the correct statistical test (T-test, Mann-Whitney, ANOVA, Kruskal-Wallis, Chi-Square, or Regression) based on the data types and groupings selected by the user.
- Time-series and categorical aggregation modules allow quick exploratory insights beyond hypothesis testing.
- Power analysis helps users estimate the minimum sample size before running tests, ensuring statistical validity.
- The system leverages OpenAI's API to translate statistical outputs into plain language insights, helping non technical stakeholders understand the results.
- Users can export all results, figures, and interpretations into a single PDF report, making it suitable for presentations and decision-making.

Overall, this project serves as both a **portfolio piece** and a **practical tool** for exploratory data analysis and hypothesis testing.  
It can be extended by adding:
- Support for multi-variable regression and interaction terms.
- Automated A/B testing dashboards.
- Integration with databases (SQL) or cloud-based storage.
- More AI-driven recommendations (e.g., automatic business action suggestions).

This notebook not only showcases technical implementation but also bridges the gap between **raw statistical analysis** and **decision-ready insights**.


